In [17]:
#pip install regex
# pip install praw

In [18]:
clear()

In [19]:
# Import libraries
import pandas as pd
import numpy as np # linear algebra
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import unicodedata
import datetime
import praw
from nltk.tokenize import RegexpTokenizer
import regex as re
from bs4 import BeautifulSoup    


from nltk.corpus import stopwords # Import the stopword list
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

from warnings import simplefilter
# Optionally turn off warnings once the models are producing good results
#   non convergence may mean the result is not perfect, but it might be good enough
# Credit to Jamie Shaffer
simplefilter("ignore", category=0)

In [20]:
# This class is used to pull data from Reddit and then create a DataFrame, it also loads the lastest csv into a dataframe

class reddit_getter:
    def __init__(self, sub):
        # This initalizes and loads all our past datasets
        self.subreddit = sub
        self.submits = 1000
        self.time_filter = 'day'
        self.reddit = praw.Reddit(client_id = 'oZRFffq6V4xYMg',
                    client_secret = 'O_p5-j5D3fYEa58nqMOwHnGZw0E',
                    username = 'the_illuminati_666',
                    user_agent = 'nothing_nefarious')
        self.dframe = pd.read_csv('../data_files/combined_reddit_data.csv')
    
    # This function can be called to pull data from Reddit, provided a submission_getter object has been created
    def get(self):
        comment_dict = {}
        subreddit = self.reddit.subreddit(self.subreddit)
        reddit_data = subreddit.top(time_filter = self.time_filter, limit = self.submits)
        print(f'Subreddit We Are Mining: {self.subreddit}, Time Filter: {self.time_filter}')
        for submission in reddit_data:
            # sometimes the submission.stickied method throws an error, we'll use a try and except in case
            try:
                if not submission.stickied:
                    comment_dict.update({submission.id : [submission.title, submission.author, subreddit.title, submission.num_comments, str(self.time_filter)]})          
            except:
                pass
        df = pd.DataFrame.from_dict(data = comment_dict, orient = 'index', columns = ['Submission Title', 'Submission Author', 'Subreddit', 'Number of Comments', 'Time Filter'])
        print('Returning DataFrame')
        return df
    

In [21]:
df_object = reddit_getter('Politics')
df = df_object.dframe
df.head()

,Submission Title,Submission Author,Subreddit,Number of Comments,Time Filter
0,Megathread: House Votes to Impeach President D...,PoliticsModeratorBot,Politics,52218,all
1,Virginia governor makes Election Day a holiday...,nclobo,Politics,4158,all
2,Video emerges showing Trump talking about cutt...,Nihilist911,Politics,4785,all
3,The Trump administration paid a bankrupt compa...,DaFunkJunkie,Politics,4809,all
4,Removed US coronavirus vaccine director files ...,grepnork,Politics,5089,all


In [22]:
# This class cleans and lemmatizes our sentences
class clean_sentences:
    def __init__(self, df):
        s = []

        df['Subreddit'] = df['Subreddit'].map({'Politics' : 1, 'Democrats: Stronger Together' : 0})
        for item in df['Submission Title']:
            
            letters = re.sub("[^a-zA-Z]", " ", item)

            words = letters.lower().split()

            stops = set(stopwords.words('english'))

            more_words = [w for w in words if w not in stops]
            
            cleaned_sentence = " ".join(more_words)

            s.append(cleaned_sentence)

        df['cleaned_submissions'] = s

        lems = WordNetLemmatizer()
        lemmed = []
        for item in df['cleaned_submissions']:
            s_temp = ""
            s1 = item.split()
            lem_r = [lems.lemmatize(s) for s in s1]
            s_temp = " ".join(lem_r)
            lemmed.append(s_temp)
            
        df['lemmed'] = lemmed
        self.dframe = df

    

In [23]:
# Create an Object
df_o = clean_sentences(df)

In [24]:
df = df_o.dframe
df.head()

,Submission Title,Submission Author,Subreddit,Number of Comments,Time Filter,cleaned_submissions,lemmed
0,Megathread: House Votes to Impeach President D...,PoliticsModeratorBot,1,52218,all,megathread house votes impeach president donal...,megathread house vote impeach president donald...
1,Virginia governor makes Election Day a holiday...,nclobo,1,4158,all,virginia governor makes election day holiday e...,virginia governor make election day holiday ex...
2,Video emerges showing Trump talking about cutt...,Nihilist911,1,4785,all,video emerges showing trump talking cutting pa...,video emerges showing trump talking cutting pa...
3,The Trump administration paid a bankrupt compa...,DaFunkJunkie,1,4809,all,trump administration paid bankrupt company zer...,trump administration paid bankrupt company zer...
4,Removed US coronavirus vaccine director files ...,grepnork,1,5089,all,removed us coronavirus vaccine director files ...,removed u coronavirus vaccine director file st...


In [25]:
# Define our variables
X = df['lemmed']
y = df['Subreddit']

In [26]:
# The shapes match
X.shape

(4636,)

In [27]:
y.shape

(4636,)

In [28]:
# Split the data into the training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.33,
                                                    stratify=y,
                                                    random_state=42)



In [29]:
X_train.isnull().sum()

0

In [30]:
# Everything looks ok
y_train.value_counts()

1    2365
0     741
Name: Subreddit, dtype: int64

In [31]:
# This class runs 4 models, and then prints their metrics
class analyzer:
    def __init__(self, X_train, y_train, X_test, y_test):
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        
    def do_it(self):
        
        # This methods runs four differant models and then prints out the results
        print(f'Starting Analysis, Be Patient... \n')
        

        print(f'Running CountVectorizer + LogisticRegression')
        print('-------------------------------------------- \n')
        pipe1 = Pipeline([
        ('cvec', CountVectorizer(stop_words = 'english')),
        ('lr', LogisticRegression())
        ])   
        
        pipe_params = {
            'cvec__max_features' : [500, 1000, 1500],
            'cvec__min_df' : [2, 3],
            'cvec__max_df' : [.9, .95],
            'cvec__ngram_range' : [(1, 1), (1, 2)],
            'lr__solver' : ['newton-cg', 'lbfgs']
            }
        gs1 = GridSearchCV(pipe1,
        pipe_params,
        cv = 5) # 5-fold cross-validation.
        
        # Fit GridSearch to training data.
        gs1.fit(self.X_train, self.y_train);
        gs1_model = gs1.best_estimator_
        preds1 = gs1_model.predict(self.X_test)
        print('1 of 4  Models Finished \n')
        print(f'I Said Be Patient!!! \n')            
            
        # Now we'll run CountVectorizer + MultinomialNB
        print(f'Running CountVectorizer + MultinomialNB')
        print('--------------------------------------- \n')
        pipe2 = Pipeline([
            ('cvec', CountVectorizer(stop_words = 'english')),
            ('mnb', MultinomialNB())
        ])

        # Set up pipeline parameters
        pipe_params2 = {
            'cvec__max_features' : [500, 1000, 1500],
            'cvec__min_df' : [2, 3],
            'cvec__max_df' : [.9, .95],
            'cvec__ngram_range' : [(1, 1), (1, 2)],
#             'mnb__alpha': [1.0, 0, .01], 
            'mnb__fit_prior': [True, False]
        }
        
        gs2 = GridSearchCV(pipe2,
        pipe_params2,
        cv = 5) # 5-fold cross-validation.
        
        # Fit GridSearch to training data.
        gs2.fit(self.X_train, self.y_train);
        gs2_model = gs2.best_estimator_
        preds2 = gs2_model.predict(self.X_test)
        print('2 of 4  Models Finished \n')


        # Now we'll run TfidfVectorizer + LogisticRegression
        print(f'Running TfidfVectorizer + LogisticRegression')
        print('-------------------------------------------- \n')
        # Set up pipeline params
        pipe3 = Pipeline([
        ('tvec', TfidfVectorizer(stop_words = 'english')),
        ('lr', LogisticRegression())
        ])
    

        # Set up pipeline params
        pipe_params = {
            'tvec__max_features': [200, 500],
            'tvec__ngram_range' : [(1, 2)],
            'lr__solver' : ['lbfgs', 'newton-cg']
        }
        
        gs3 = GridSearchCV(pipe3, # what object are we optimizing?
        pipe_params, # what parameters values are we searching?
        cv = 5) # 5-fold cross-validation.
        
        # Fit GridSearch to training data.
        gs3.fit(self.X_train, self.y_train);
        gs3_model = gs3.best_estimator_
        preds3 = gs3_model.predict(self.X_test)
        print('3 of 4  Models Finished \n')


        # Now we'll run TfidfVectorizer + MultinomialNB
        print(f'Running TfidfVectorizer + MultinomialNB')
        print('--------------------------------------- \n')
        # Set up pipeline params
        pipe4 = Pipeline([
        ('tvec', TfidfVectorizer(stop_words = 'english')),
        ('mnb', MultinomialNB())
        ])


        # Set up pipeline params
        pipe_params = {
            'tvec__max_features': [200, 500],
            'tvec__ngram_range' : [(1, 2)],
#             'mnb__alpha': [10.0, 0, .01], 
            'mnb__fit_prior': [True, False]
        }
        
        gs4 = GridSearchCV(pipe4,
        pipe_params,
        cv = 5) # 5-fold cross-validation.
        
        # Fit GridSearch to training data.
        gs4.fit(self.X_train, self.y_train);
        gs4_model = gs4.best_estimator_
        preds4 = gs4_model.predict(self.X_test)
        print('4 of 4  Models Finished \n')
 
        
        
        
        # ---------------------------------------
        # Output
        print('Model Metrics:')
        print('-------------- \n')
        
        print(f'CountVectorizer & LogisticRegression \n')
        print(f'Best Score: {gs1.best_score_} \n')

        print(f'X_train, y_train Score: {gs1_model.score(self.X_train, self.y_train)}')
        print(f'X_test, y_test Score: {gs1_model.score(self.X_test, self.y_test)} \n')

        cm = confusion_matrix(self.y_test, preds1)
        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]
        print(f'True Positives: {TP}, True Negatives: {TN}, False Positives: {FP}, False Negatives: {FN} \n')

        print(f'{classification_report(self.y_test, preds1)} \n')

        print('--------------------------------- \n')

        
        print(f'CountVectorizer & MultinomialNB \n')

        print(f'Best Score: {gs2.best_score_} \n')

        print(f'X_train, y_train Score: {gs2_model.score(self.X_train, self.y_train)}')
        print(f'X_test, y_test Score: {gs2_model.score(self.X_test, self.y_test)} \n')

        cm = confusion_matrix(self.y_test, preds2)
        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]
        print(f'True Positives: {TP}, True Negatives: {TN}, False Positives: {FP}, False Negatives: {FN} \n')

        print(f'{classification_report(self.y_test, preds2)} \n')

        print('------------------------------------- \n')

        
        print(f'TfidfVectorizer & LogisticRegression \n')

        print(f'Best Score: {gs3.best_score_} \n')

        print(f'X_train, y_train Score: {gs3_model.score(self.X_train, self.y_train)}')
        print(f'X_test, y_test Score: {gs3_model.score(self.X_test, self.y_test)} \n')

        cm = confusion_matrix(self.y_test, preds3)
        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]
        print(f'True Positives: {TP}, True Negatives: {TN}, False Positives: {FP}, False Negatives: {FN} \n')

        print(f'{classification_report(self.y_test, preds3)} \n')

        print('-------------------------------- \n')
        
        print(f'TfidfVectorizer & MultinomialNB \n')

        print(f'Best Score: {gs4.best_score_} \n')

        print(f'X_train, y_train Score: {gs4_model.score(self.X_train, self.y_train)}')
        print(f'X_test, y_test Score: {gs4_model.score(self.X_test, self.y_test)} \n')
        cm = confusion_matrix(self.y_test, preds4)
        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]
        print(f'True Positives: {TP}, True Negatives: {TN}, False Positives: {FP}, False Negatives: {FN} \n')

        print(f'{classification_report(self.y_test, preds4)} \n')
        print('------------------ \n')

        print('>----- Analysis Complete, Thank You For Flying With Us! -------> \n')




In [32]:
# Create an analyzer object
a = analyzer(X_train, y_train, X_test, y_test)

In [25]:
# Call the analysis method and enjoy the results
a.do_it()

Starting Analysis, Be Patient... 

Running CountVectorizer + LogisticRegression
-------------------------------------------- 

1 of 4  Models Finished 

I Said Be Patient!!! 

Running CountVectorizer + MultinomialNB
--------------------------------------- 

2 of 4  Models Finished 

Running TfidfVectorizer + LogisticRegression
-------------------------------------------- 

3 of 4  Models Finished 

Running TfidfVectorizer + MultinomialNB
--------------------------------------- 

4 of 4  Models Finished 

Model Metrics:
-------------- 

CountVectorizer & LogisticRegression 

Best Score: 0.7875053720013876 

X_train, y_train Score: 0.8950418544752092
X_test, y_test Score: 0.7862745098039216 

True Positives: 1092, True Negatives: 111, False Positives: 254, False Negatives: 73 

              precision    recall  f1-score   support

           0       0.60      0.30      0.40       365
           1       0.81      0.94      0.87      1165

    accuracy                           0.79      